In [28]:
import pandas as pd
import argparse
import numpy as np
import json
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, BertConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BertTokenizer, BertForMaskedLM, DataCollatorForTokenClassification
from adapters import AutoAdapterModel
from adapters import AdapterConfig
from adapters import AdapterTrainer
from evaluate import load



In [ ]:
#the language tags we are using are: bg ms ne jv mt ug bo si

# Bulgarian, Indonesian, Nepali, Javanese,  Maltese, Uyghur, Tibetan, Sinhala Respectively. 

# for the specific language, replace the tag before "..."/validation , "..."/train, and "..."/test with the correct language tag



In [3]:
"""


def tokenize_adjust_labels(all_samples_per_split):
        tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], padding=True, truncation=True, is_split_into_words=True)
        # tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
        # so the new keys [input_ids, labels (after adjustment)]
        # can be added to the datasets dict for each train test validation split
        total_adjusted_labels = []
        for k in range(0, len(tokenized_samples["input_ids"])):
            prev_wid = -1
            word_ids_list = tokenized_samples.word_ids(batch_index=k)
            existing_label_ids = all_samples_per_split["ner_tags"][k]
            i = -1
            adjusted_label_ids = []
        
            for wid in word_ids_list:
                if(wid is None):
                    adjusted_label_ids.append(-100)
                elif(wid!=prev_wid):
                    i = i + 1
                    adjusted_label_ids.append(existing_label_ids[i])
                    prev_wid = wid
                else:
                    label_name = label_names[existing_label_ids[i]]
                    adjusted_label_ids.append(existing_label_ids[i])
                
            total_adjusted_labels.append(adjusted_label_ids)
        tokenized_samples["labels"] = total_adjusted_labels
        tokenized_samples["labels"] = [list(map(int, x)) for x in tokenized_samples["labels"]]



dataset = load_dataset("wikiann", 'bg')

label_names = dataset["train"].features["ner_tags"].feature.names
print(label_names)
id2label = {id_: label for id_, label in enumerate(label_names)}
label2id = {label: id_ for id_, label in enumerate(label_names)}

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

# prepare model
config = AutoConfig.from_pretrained("bert-base-multilingual-cased", id2label=id2label, label2id=label2id)
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased", config=config)


print("final processed data set")
print(tokenized_dataset["train"][0])'

#creating a dataset collator (this is what facilitates the masked language modelling functionality)
"""


seeing what load_dataset looks like
{'tokens': ['виж', 'Джон', 'Уилям', 'Стрът'], 'ner_tags': [0, 1, 2, 2], 'langs': ['bg', 'bg', 'bg', 'bg'], 'spans': ['PER: Джон Уилям Стрът']}
after load dataset
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


Map: 100%|██████████| 20000/20000 [00:00<00:00, 26247.43 examples/s]
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initial

final processed data set
{'tokens': ['виж', 'Джон', 'Уилям', 'Стрът'], 'ner_tags': [0, 1, 2, 2], 'langs': ['bg', 'bg', 'bg', 'bg'], 'spans': ['PER: Джон Уилям Стрът']}


In [18]:
#I think this is the MLM pre processing for the data set: 

#load dataset
dataset = load_dataset("wikiann", 'bg')

#extracting "ner_tags" names
label_names = dataset["train"].features["ner_tags"].feature.names
#using tokenize_function. I'm splitting into words because that's what the paper code does


#tokenizing function and pre processing it to feed into mBERT. 
def tokenize_function(examples):
    return tokenizer(examples["tokens"], truncation=True, padding=True,is_split_into_words=True, )

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["tokens", "ner_tags", "langs", "spans"])


id2label = {id_: label for id_, label in enumerate(label_names)}
label2id = {label: id_ for id_, label in enumerate(label_names)}

# prepare model
config = AutoConfig.from_pretrained("bert-base-multilingual-cased", id2label=id2label, label2id=label2id)
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased", config=config)


#print(tokenized_dataset["train"].features)

#putting model onto my mac's 'mps' system

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

model = model.to(device)

print(tokenized_datasets["train"].features)
print(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
mps


In [19]:

from accelerate import Accelerator
# uniform way of tracking our metrics: 
# Training arguments
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    weight_decay=0.01,
    output_dir="./training_output",
    overwrite_output_dir=True,
    save_total_limit=1,
    load_best_model_at_end=True,
    save_only_model=True,
)
#trainer values. (I'm using regular trainer, not adaptaTrainer). Data collator necessary for masked language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        #compute_metrics=lambda p: compute_metrics(p, label_names)
    )




trainer.train()
model.save_pretrained("./trained_mbert")
tokenizer.save_pretrained("./trained_mbert")

/Users/sohumbelagode/Desktop/nlp/research project/code/myCode/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 33%|███▎      | 313/939 [2:03:11<3:30:39, 20.19s/it]

{'eval_loss': 8.333383560180664, 'eval_runtime': 268.3144, 'eval_samples_per_second': 37.27, 'eval_steps_per_second': 0.585, 'epoch': 1.0}


 53%|█████▎    | 500/939 [2:48:21<1:12:19,  9.88s/it]  

{'loss': 7.3594, 'grad_norm': nan, 'learning_rate': 4.6751863684771034e-05, 'epoch': 1.6}


 67%|██████▋   | 626/939 [3:10:27<42:49,  8.21s/it]

...

100%|██████████| 939/939 [4:02:34<00:00,  7.56s/it]

{'eval_loss': nan, 'eval_runtime': 105.8411, 'eval_samples_per_second': 94.481, 'eval_steps_per_second': 1.483, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].
100%|██████████| 939/939 [4:02:36<00:00, 15.50s/it]


{'train_runtime': 14556.2767, 'train_samples_per_second': 4.122, 'train_steps_per_second': 0.065, 'train_loss': 3.9187362719648564, 'epoch': 3.0}


('./trained_mbert/tokenizer_config.json',
 './trained_mbert/special_tokens_map.json',
 './trained_mbert/vocab.txt',
 './trained_mbert/added_tokens.json',
 './trained_mbert/tokenizer.json')

In [24]:
def tokenize_adjust_labels(all_samples_per_split):
        tokenized_samples = tokenizerNER.batch_encode_plus(all_samples_per_split["tokens"], padding=True, truncation=True, is_split_into_words=True)
        # tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
        # so the new keys [input_ids, labels (after adjustment)]
        # can be added to the datasets dict for each train test validation split
        total_adjusted_labels = []
        for k in range(0, len(tokenized_samples["input_ids"])):
            prev_wid = -1
            word_ids_list = tokenized_samples.word_ids(batch_index=k)
            existing_label_ids = all_samples_per_split["ner_tags"][k]
            i = -1
            adjusted_label_ids = []
        
            for wid in word_ids_list:
                if(wid is None):
                    adjusted_label_ids.append(-100)
                elif(wid!=prev_wid):
                    i = i + 1
                    adjusted_label_ids.append(existing_label_ids[i])
                    prev_wid = wid
                else:
                    label_name = label_names[existing_label_ids[i]]
                    adjusted_label_ids.append(existing_label_ids[i])
                
            total_adjusted_labels.append(adjusted_label_ids)
        tokenized_samples["labels"] = total_adjusted_labels
        tokenized_samples["labels"] = [list(map(int, x)) for x in tokenized_samples["labels"]]

        return tokenized_samples



datasetNER = load_dataset("wikiann", 'bg')
label_names = datasetNER["train"].features["ner_tags"].feature.names
print(label_names)
id2label = {id_: label for id_, label in enumerate(label_names)}
label2id = {label: id_ for id_, label in enumerate(label_names)}

tokenizerNER = AutoTokenizer.from_pretrained("./trained_mbert")
NER_tokenized_dataset = datasetNER.map(tokenize_adjust_labels, batched=True)

# prepare model


previous_mlm_model = BertForMaskedLM.from_pretrained("./trained_mbert")

# Extract the base BERT model
base_model = previous_mlm_model.bert

# Create a token classification model
NERconfig = BertConfig.from_pretrained("./trained_mbert")
NERconfig.num_labels = 7  # Set the number of labels for token classification

NERconfig.num_labels = len(label_names)  # Automatically set number of labels
NERconfig.id2label = id2label
NERconfig.label2id = label2id

NER_token_classification_model = AutoModelForTokenClassification.from_config(NERconfig)

# Assign the base BERT model to the token classification model
NER_token_classification_model.bert = base_model

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

NER_token_classification_model = NER_token_classification_model.to(device)

print("final processed data set for NER")
print(NER_tokenized_dataset["train"][0])




['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


Map: 100%|██████████| 20000/20000 [00:01<00:00, 16491.72 examples/s]


final processed data set for NER
{'tokens': ['виж', 'Джон', 'Уилям', 'Стрът'], 'ner_tags': [0, 1, 2, 2], 'langs': ['bg', 'bg', 'bg', 'bg'], 'spans': ['PER: Джон Уилям Стрът'], 'input_ids': [101, 88504, 12025, 21499, 528, 13460, 27429, 526, 46672, 13368, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
#how the paper computes metrics: 
from seqeval.metrics import classification_report

def compute_metrics(p, label_names):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    metric = load("seqeval")
    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
        if k not in flattened_results.keys():
            flattened_results[k+"_f1"] = results[k]["f1"]
    return flattened_results


# Training arguments
ner_training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    weight_decay=0.01,
    output_dir="./NER_training_output",
    overwrite_output_dir=True,
    save_total_limit=1,
    load_best_model_at_end=True,
    save_only_model=True,
)
#trainer values. (I'm using regular trainer, not adaptaTrainer)
ner_trainer = Trainer(
        model=NER_token_classification_model,
        args=ner_training_args,
        train_dataset=NER_tokenized_dataset["train"],
        eval_dataset=NER_tokenized_dataset["validation"],
        
        data_collator=DataCollatorForTokenClassification(tokenizerNER),
        tokenizer=tokenizerNER,
        compute_metrics=lambda p: compute_metrics(p, label_names)
    )




ner_trainer.train()
#saving model and tokenizer
NER_token_classification_model.save_pretrained("./NER_trained_mbert")
tokenizerNER.save_pretrained("./NER_trained_mbert")

 33%|███▎      | 313/939 [23:11<46:23,  4.45s/it]

 33%|███▎      | 313/939 [11:12<19:19,  1.85s/it]






























































































































































                                                 
                                             
 33%|███▎      | 313/939 [11:56<19:19,  1.85s/it]


{'eval_loss': 0.21806278824806213, 'eval_overall_precision': 0.8976061317572126, 'eval_overall_recall': 0.9165036236876188, 'eval_overall_f1': 0.9069564506012926, 'eval_overall_accuracy': 0.9555929897307979, 'eval_LOC_f1': 0.9378401257709518, 'eval_ORG_f1': 0.8467210697079933, 'eval_PER_f1': 0.9171853041557921, 'eval_runtime': 43.9653, 'eval_samples_per_second': 227.452, 'eval_steps_per_second': 3.571, 'epoch': 1.0}


                                                   
 53%|█████▎    | 500/939 [18:39<16:00,  2.19s/it]

{'loss': 0.051, 'grad_norm': 2.3674802780151367, 'learning_rate': 4.6751863684771034e-05, 'epoch': 1.6}


 67%|██████▋   | 626/939 [23:11<10:07,  1.94s/it]






























































































































































                                                 
                                             
 67%|██████▋   | 626/939 [23:53<10:07,  1.94s/it]


...

100%|██████████| 939/939 [35:18<00:00,  1.82s/it]  






























































































































































                                                 
                                             
100%|██████████| 939/939 [36:01<00:00,  1.82s/it]


{'eval_loss': 0.19064943492412567, 'eval_overall_precision': 0.9186303688938645, 'eval_overall_recall': 0.9270853426658215, 'eval_overall_f1': 0.9228384902348036, 'eval_overall_accuracy': 0.9630762253393372, 'eval_LOC_f1': 0.9475979850104436, 'eval_ORG_f1': 0.8722650754378305, 'eval_PER_f1': 0.9348123385503724, 'eval_runtime': 42.5562, 'eval_samples_per_second': 234.984, 'eval_steps_per_second': 3.689, 'epoch': 3.0}


                                                 
100%|██████████| 939/939 [36:03<00:00,  2.30s/it]


{'train_runtime': 2163.2549, 'train_samples_per_second': 27.736, 'train_steps_per_second': 0.434, 'train_loss': 0.03983610658980787, 'epoch': 3.0}


('./NER_trained_mbert/tokenizer_config.json',
 './NER_trained_mbert/special_tokens_map.json',
 './NER_trained_mbert/vocab.txt',
 './NER_trained_mbert/added_tokens.json',
 './NER_trained_mbert/tokenizer.json')

In [33]:
# test model
test_results = ner_trainer.evaluate(eval_dataset=NER_tokenized_dataset["test"])
output_file_path = os.path.join("./test_metrics.json")
with open(output_file_path, "w") as f:
    json.dump(test_results, f)

100%|██████████| 157/157 [00:44<00:00,  3.54it/s]


In [41]:
#this is meant to be a training loop for the other languages. It first uses masked language modeling to train on new domain, 
# then attatches a prediction head (a linear classification layer) and trains that too: 

totalLanguages = ['ne' ,'jv', 'mt', 'ug' ,'bo' ,'ms', 'si']

for language in totalLanguages:


    ## initial data collection loop 
    #load dataset
    dataset = load_dataset("wikiann", language)

    #extracting "ner_tags" names
    label_names = dataset["train"].features["ner_tags"].feature.names
    #using tokenize_function. I'm splitting into words because that's what the paper code does

    def tokenize_function(examples):
        return tokenizer(examples["tokens"], truncation=True, padding=True,is_split_into_words=True, )

    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=True)
    tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["tokens", "ner_tags", "langs", "spans"])


    id2label = {id_: label for id_, label in enumerate(label_names)}
    label2id = {label: id_ for id_, label in enumerate(label_names)}

    # prepare model
    config = AutoConfig.from_pretrained("bert-base-multilingual-cased", id2label=id2label, label2id=label2id)
    model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased", config=config)


    #print(tokenized_dataset["train"].features)

    #putting model onto my mac's 'mps' system

    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

    model = model.to(device)
    training_args = TrainingArguments(
        learning_rate=1e-4,
        num_train_epochs=3,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        weight_decay=0.01,
        output_dir="./training_output/" + language,
        overwrite_output_dir=True,
        save_total_limit=1,
        load_best_model_at_end=True,
        save_only_model=True,
    )
    #trainer values. (I'm using regular trainer, not adaptaTrainer)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
    trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"],
            data_collator=data_collator,
            tokenizer=tokenizer,
            #compute_metrics=lambda p: compute_metrics(p, label_names)
        )




    trainer.train()
    model.save_pretrained("./trained_mbert/" + language)
    tokenizer.save_pretrained("./trained_mbert/" + language)

    def tokenize_adjust_labels(all_samples_per_split):
            tokenized_samples = tokenizerNER.batch_encode_plus(all_samples_per_split["tokens"], padding=True, truncation=True, is_split_into_words=True)
            # tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
            # so the new keys [input_ids, labels (after adjustment)]
            # can be added to the datasets dict for each train test validation split
            total_adjusted_labels = []
            for k in range(0, len(tokenized_samples["input_ids"])):
                prev_wid = -1
                word_ids_list = tokenized_samples.word_ids(batch_index=k)
                existing_label_ids = all_samples_per_split["ner_tags"][k]
                i = -1
                adjusted_label_ids = []
            
                for wid in word_ids_list:
                    if(wid is None):
                        adjusted_label_ids.append(-100)
                    elif(wid!=prev_wid):
                        i = i + 1
                        adjusted_label_ids.append(existing_label_ids[i])
                        prev_wid = wid
                    else:
                        label_name = label_names[existing_label_ids[i]]
                        adjusted_label_ids.append(existing_label_ids[i])
                    
                total_adjusted_labels.append(adjusted_label_ids)
            tokenized_samples["labels"] = total_adjusted_labels
            tokenized_samples["labels"] = [list(map(int, x)) for x in tokenized_samples["labels"]]

            return tokenized_samples



    datasetNER = load_dataset("wikiann", language)
    label_names = datasetNER["train"].features["ner_tags"].feature.names
    id2label = {id_: label for id_, label in enumerate(label_names)}
    label2id = {label: id_ for id_, label in enumerate(label_names)}

    tokenizerNER = AutoTokenizer.from_pretrained("./trained_mbert/" + language)
    NER_tokenized_dataset = datasetNER.map(tokenize_adjust_labels, batched=True)

    # prepare model


    previous_mlm_model = BertForMaskedLM.from_pretrained("./trained_mbert/" + language)

    # Extract the base BERT model
    base_model = previous_mlm_model.bert

    # Create a token classification model
    NERconfig = BertConfig.from_pretrained("./trained_mbert/" + language)
    NERconfig.num_labels = 7  # Set the number of labels for token classification

    NERconfig.num_labels = len(label_names)  # Automatically set number of labels
    NERconfig.id2label = id2label
    NERconfig.label2id = label2id

    NER_token_classification_model = AutoModelForTokenClassification.from_config(NERconfig)

    # Assign the base BERT model to the token classification model
    NER_token_classification_model.bert = base_model

    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

    NER_token_classification_model = NER_token_classification_model.to(device)




    def compute_metrics(p, label_names):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)
        true_predictions = [
            [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        metric = load("seqeval")
        results = metric.compute(predictions=true_predictions, references=true_labels)
        flattened_results = {
            "overall_precision": results["overall_precision"],
            "overall_recall": results["overall_recall"],
            "overall_f1": results["overall_f1"],
            "overall_accuracy": results["overall_accuracy"],
        }
        for k in results.keys():
            if k not in flattened_results.keys():
                flattened_results[k+"_f1"] = results[k]["f1"]
        return flattened_results


    # Training arguments
    ner_training_args = TrainingArguments(
        learning_rate=1e-4,
        num_train_epochs=3,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        weight_decay=0.01,
        output_dir="./NER_training_output/" + language,
        overwrite_output_dir=True,
        save_total_limit=1,
        load_best_model_at_end=True,
        save_only_model=True,
    )
    #trainer values. (I'm using regular trainer, not adaptaTrainer)
    ner_trainer = Trainer(
            model=NER_token_classification_model,
            args=ner_training_args,
            train_dataset=NER_tokenized_dataset["train"],
            eval_dataset=NER_tokenized_dataset["validation"],
            
            data_collator=DataCollatorForTokenClassification(tokenizerNER),
            tokenizer=tokenizerNER,
            compute_metrics=lambda p: compute_metrics(p, label_names)
        )




    ner_trainer.train()
    NER_token_classification_model.save_pretrained("./NER_trained_mbert/" + language)
    tokenizerNER.save_pretrained("./NER_trained_mbert/" + language)


    test_results = ner_trainer.evaluate(eval_dataset=NER_tokenized_dataset["test"])
    results_path = "./test_metrics_" + language + ".json"
    output_file_path = os.path.join(results_path)
    with open(output_file_path, "w") as f:
        json.dump(test_results, f)



    

    


Map: 100%|██████████| 100/100 [00:00<00:00, 12462.65 examples/s]
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/sohumbelagode/Desktop/nlp/research project/code/myCode/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use 

{'eval_loss': 2.424934148788452, 'eval_runtime': 1.8765, 'eval_samples_per_second': 53.289, 'eval_steps_per_second': 1.066, 'epoch': 1.0}


 67%|██████▋   | 4/6 [00:11<00:04,  2.28s/it]

{'eval_loss': 2.168142318725586, 'eval_runtime': 1.9714, 'eval_samples_per_second': 50.727, 'eval_steps_per_second': 1.015, 'epoch': 2.0}


100%|██████████| 6/6 [00:21<00:00,  2.55s/it]

...

100%|██████████| 6/6 [00:04<00:00,  1.72it/s]

{'eval_loss': 1.2496193647384644, 'eval_overall_precision': 0.08974358974358974, 'eval_overall_recall': 0.056910569105691054, 'eval_overall_f1': 0.06965174129353234, 'eval_overall_accuracy': 0.6143884892086331, 'eval_LOC_f1': 0.0, 'eval_ORG_f1': 0.12612612612612611, 'eval_PER_f1': 0.0, 'eval_runtime': 0.4489, 'eval_samples_per_second': 222.781, 'eval_steps_per_second': 4.456, 'epoch': 3.0}


100%|██████████| 6/6 [00:04<00:00,  1.28it/s]


{'train_runtime': 4.6894, 'train_samples_per_second': 63.974, 'train_steps_per_second': 1.279, 'train_loss': 1.4594939549763997, 'epoch': 3.0}


100%|██████████| 2/2 [00:01<00:00,  1.82it/s]


In [40]:
language = "./test_metrics_" + "ne" + ".json"
output_file_path = os.path.join(language)
with open(output_file_path, "w") as f:
    json.dump(test_results, f)